In [1]:
# allow the notebook to access the parent directory so we can import the other modules
# https://stackoverflow.com/a/35273613
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Data Preparation
-----

### Constants and Folder Paths

In [15]:
import os
dataset_folder_path = os.path.join("..", "files", "dataset")
RANDOM_STATE = 42

### Load Data and Split into *Test*, *Train/Valid*

In [3]:
from data.DataSet import DataSet
dataset = DataSet()
# dataset.load(dataset_folder_path, test_set_percentage=0.3333, validation_set_percentage=0.3333)
dataset.load(dataset_folder_path, test_set_percentage=0, validation_set_percentage=0)

In [4]:
print(len(dataset.train_data))
print(len(dataset.test_data))

3600
0


### Data Preprocessing

In [5]:
NUM_SAMPLES = 50
from utils.preprocessing import *
from functools import partial
dataset.apply(apply_mean_centering)
dataset.apply(apply_unit_distance_normalization)
#dataset.apply(partial(normalize_pressure_value, max_pressure_val=512))
dataset.apply(partial(spline_interpolate_and_resample, num_samples=NUM_SAMPLES))
dataset.expand(reverse_digit_sequence)
# dataset.apply(lambda digit: convert_xy_to_derivative(digit, normalize=False))
#dataset.apply(partial(convert_xy_to_derivative, normalize=True))

In [6]:
print(len(dataset.train_data))
print(len(dataset.test_data))

7200
0


### Setup Dataset, don't split, don't onehot encode, since we will perform cross validation

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
X_train_orig = np.array(dataset.train_data)
X_train_orig.shape

(7200, 50, 2)

In [9]:
Y_train = np.array(dataset.train_labels)
Y_train.shape

(7200,)

# X Axis Only:
-------

#### Select only X axis values

In [12]:
X_train = X_train_orig[:, :, 0].reshape(X_train_orig.shape[0], X_train_orig.shape[1], 1)
X_train.shape

(7200, 50, 1)

----------
## **Regularized Deep GRU**
----------

In [21]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 300

from models.regularized_deep_gru import RegularizedDeepGRU
from utils.evaluation import cross_validate_model

mymodel = RegularizedDeepGRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/20
4800/4800 [==============================] - 11s 2ms/step - loss: 2.1945 - categorical_accuracy: 0.1908 - val_loss: 2.0497 - val_categorical_accuracy: 0.2467
Epoch 2/20
4800/4800 [==============================] - 10s 2ms/step - loss: 1.9984 - categorical_accuracy: 0.2775 - val_loss: 2.0123 - val_categorical_accuracy: 0.3117
Epoch 3/20
4800/4800 [==============================] - 10s 2ms/step - loss: 2.0196 - categorical_accuracy: 0.2812 - val_loss: 1.9022 - val_categorical_accuracy: 0.3021
Epoch 4/20
4800/4800 [==============================] - 10s 2ms/step - loss: 1.8495 - categorical_accuracy: 0.3427 - val_loss: 1.8314 - val_categorical_accuracy: 0.3496
Epoch 5/20
4800/4800 [==============================] - 10s 2ms/step - loss: 1.5966 - categorical_accuracy: 0.4310 - val_loss: 1.6240 - val_categorical_accuracy: 0.4087
Epoch 6/20
4800/4800 [===============

KeyboardInterrupt: 

----------
## **Regularized 64 GRU**
----------

In [22]:
PARAM_NUM_EPOCHS = 80
PARAM_BATCH_SIZE = 300

from models.regularized_64_gru import Regularized64GRU
from utils.evaluation import cross_validate_model

mymodel = Regularized64GRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/80
4800/4800 [==============================] - 2s 517us/step - loss: 2.2622 - categorical_accuracy: 0.1519 - val_loss: 2.2352 - val_categorical_accuracy: 0.1608
Epoch 2/80
4800/4800 [==============================] - 1s 258us/step - loss: 2.1700 - categorical_accuracy: 0.1923 - val_loss: 2.1269 - val_categorical_accuracy: 0.2196
Epoch 3/80
4800/4800 [==============================] - 1s 262us/step - loss: 2.0426 - categorical_accuracy: 0.2567 - val_loss: 2.0748 - val_categorical_accuracy: 0.2696
Epoch 4/80
4800/4800 [==============================] - 1s 256us/step - loss: 1.9029 - categorical_accuracy: 0.3104 - val_loss: 1.7880 - val_categorical_accuracy: 0.3463
Epoch 5/80
4800/4800 [==============================] - 1s 257us/step - loss: 1.7525 - categorical_accuracy: 0.3577 - val_loss: 1.6450 - val_categorical_accuracy: 0.4021
Epoch 6/80
4800/4800 [==========

4800/4800 [==============================] - 1s 260us/step - loss: 0.2661 - categorical_accuracy: 0.9206 - val_loss: 0.6377 - val_categorical_accuracy: 0.8508
Epoch 49/80
4800/4800 [==============================] - 1s 254us/step - loss: 0.2614 - categorical_accuracy: 0.9198 - val_loss: 0.6320 - val_categorical_accuracy: 0.8517
Epoch 50/80
4800/4800 [==============================] - 1s 254us/step - loss: 0.2638 - categorical_accuracy: 0.9156 - val_loss: 0.6334 - val_categorical_accuracy: 0.8492
Epoch 51/80
4800/4800 [==============================] - 1s 255us/step - loss: 0.2539 - categorical_accuracy: 0.9223 - val_loss: 0.6431 - val_categorical_accuracy: 0.8525
Epoch 52/80
4800/4800 [==============================] - 1s 256us/step - loss: 0.2294 - categorical_accuracy: 0.9319 - val_loss: 0.6422 - val_categorical_accuracy: 0.8554
Epoch 53/80
4800/4800 [==============================] - 1s 254us/step - loss: 0.2293 - categorical_accuracy: 0.9304 - val_loss: 0.6280 - val_categorical_acc

Epoch 15/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.7637 - categorical_accuracy: 0.7485 - val_loss: 0.7960 - val_categorical_accuracy: 0.7458
Epoch 16/80
4800/4800 [==============================] - 1s 259us/step - loss: 0.6977 - categorical_accuracy: 0.7702 - val_loss: 0.7782 - val_categorical_accuracy: 0.7504
Epoch 17/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.6703 - categorical_accuracy: 0.7792 - val_loss: 0.7587 - val_categorical_accuracy: 0.7633
Epoch 18/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.6486 - categorical_accuracy: 0.7948 - val_loss: 0.7537 - val_categorical_accuracy: 0.7658
Epoch 19/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.6199 - categorical_accuracy: 0.7944 - val_loss: 0.7197 - val_categorical_accuracy: 0.7788
Epoch 20/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.6054 - categorical_accuracy: 0.8006 - val_loss: 0.7381 - val_cat

Epoch 63/80
4800/4800 [==============================] - 1s 255us/step - loss: 0.2269 - categorical_accuracy: 0.9327 - val_loss: 0.6478 - val_categorical_accuracy: 0.8458
Epoch 64/80
4800/4800 [==============================] - 1s 261us/step - loss: 0.2144 - categorical_accuracy: 0.9325 - val_loss: 0.6965 - val_categorical_accuracy: 0.8358
Epoch 65/80
4800/4800 [==============================] - 1s 261us/step - loss: 0.2186 - categorical_accuracy: 0.9308 - val_loss: 0.6826 - val_categorical_accuracy: 0.8433
Epoch 66/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.2172 - categorical_accuracy: 0.9325 - val_loss: 0.6823 - val_categorical_accuracy: 0.8383
Epoch 67/80
4800/4800 [==============================] - 1s 260us/step - loss: 0.2138 - categorical_accuracy: 0.9358 - val_loss: 0.7204 - val_categorical_accuracy: 0.8321
Epoch 68/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.2234 - categorical_accuracy: 0.9310 - val_loss: 0.6613 - val_cat

Epoch 30/80
4800/4800 [==============================] - 1s 259us/step - loss: 0.4419 - categorical_accuracy: 0.8675 - val_loss: 0.6854 - val_categorical_accuracy: 0.8042
Epoch 31/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.4204 - categorical_accuracy: 0.8621 - val_loss: 0.6759 - val_categorical_accuracy: 0.8112
Epoch 32/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.4111 - categorical_accuracy: 0.8746 - val_loss: 0.7005 - val_categorical_accuracy: 0.8008
Epoch 33/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.3968 - categorical_accuracy: 0.8806 - val_loss: 0.6562 - val_categorical_accuracy: 0.8188
Epoch 34/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.3965 - categorical_accuracy: 0.8794 - val_loss: 0.6372 - val_categorical_accuracy: 0.8163
Epoch 35/80
4800/4800 [==============================] - 1s 260us/step - loss: 0.4011 - categorical_accuracy: 0.8744 - val_loss: 0.6116 - val_cat

Epoch 78/80
4800/4800 [==============================] - 1s 265us/step - loss: 0.2068 - categorical_accuracy: 0.9394 - val_loss: 0.6188 - val_categorical_accuracy: 0.8371
Epoch 79/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.1921 - categorical_accuracy: 0.9415 - val_loss: 0.6359 - val_categorical_accuracy: 0.8362
Epoch 80/80
2400/2400 [==============================] - 1s 489us/step
categorical_accuracy: 84.21%
84.36% (+/- 0.16%)


[84.583333333333329, 84.291666666666671, 84.208333333333329]

# Y Axis Only:
-------

#### Select only X axis values

In [16]:
X_train = X_train_orig[:, :, 1].reshape(X_train_orig.shape[0], X_train_orig.shape[1], 1)
X_train.shape

(7200, 50, 1)

----------
## **Regularized Deep GRU**
----------

In [17]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 300

from models.regularized_deep_gru import RegularizedDeepGRU
from utils.evaluation import cross_validate_model

mymodel = RegularizedDeepGRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)

Using TensorFlow backend.



....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/40
4800/4800 [==============================] - 4s 912us/step - loss: 2.2028 - categorical_accuracy: 0.1706 - val_loss: 2.0603 - val_categorical_accuracy: 0.1846
Epoch 2/40
4800/4800 [==============================] - 3s 660us/step - loss: 1.9810 - categorical_accuracy: 0.2435 - val_loss: 1.8168 - val_categorical_accuracy: 0.3667
Epoch 3/40
4800/4800 [==============================] - 3s 660us/step - loss: 1.7198 - categorical_accuracy: 0.3373 - val_loss: 1.6276 - val_categorical_accuracy: 0.4113
Epoch 4/40
4800/4800 [==============================] - 3s 665us/step - loss: 1.5734 - categorical_accuracy: 0.4065 - val_loss: 1.4838 - val_categorical_accuracy: 0.4850
Epoch 5/40
4800/4800 [==============================] - 3s 679us/step - loss: 1.3752 - categorical_accuracy: 0.4837 - val_loss: 1.4284 - val_categorical_accuracy: 0.4950
Epoch 6/40
4800/4800 [==========

4800/4800 [==============================] - 3s 680us/step - loss: 1.1614 - categorical_accuracy: 0.5892 - val_loss: 1.0361 - val_categorical_accuracy: 0.6133
Epoch 8/40
4800/4800 [==============================] - 3s 658us/step - loss: 1.0204 - categorical_accuracy: 0.6406 - val_loss: 0.8220 - val_categorical_accuracy: 0.6937
Epoch 9/40
4800/4800 [==============================] - 3s 703us/step - loss: 1.0076 - categorical_accuracy: 0.6385 - val_loss: 0.7537 - val_categorical_accuracy: 0.7300
Epoch 10/40
4800/4800 [==============================] - 3s 706us/step - loss: 0.8474 - categorical_accuracy: 0.7119 - val_loss: 0.6771 - val_categorical_accuracy: 0.7713
Epoch 11/40
4800/4800 [==============================] - 3s 706us/step - loss: 0.7771 - categorical_accuracy: 0.7344 - val_loss: 0.6307 - val_categorical_accuracy: 0.7779
Epoch 12/40
4800/4800 [==============================] - 3s 702us/step - loss: 0.6961 - categorical_accuracy: 0.7640 - val_loss: 0.5559 - val_categorical_accur

Epoch 14/40
4800/4800 [==============================] - 3s 699us/step - loss: 0.5759 - categorical_accuracy: 0.8015 - val_loss: 0.7315 - val_categorical_accuracy: 0.7608
Epoch 15/40
4800/4800 [==============================] - 3s 710us/step - loss: 0.5871 - categorical_accuracy: 0.7898 - val_loss: 0.7173 - val_categorical_accuracy: 0.7696
Epoch 16/40
4800/4800 [==============================] - 3s 708us/step - loss: 0.4961 - categorical_accuracy: 0.8281 - val_loss: 0.6586 - val_categorical_accuracy: 0.7888
Epoch 17/40
4800/4800 [==============================] - 3s 720us/step - loss: 0.4498 - categorical_accuracy: 0.8473 - val_loss: 0.6525 - val_categorical_accuracy: 0.7892
Epoch 18/40
4800/4800 [==============================] - 4s 731us/step - loss: 0.4306 - categorical_accuracy: 0.8485 - val_loss: 0.6442 - val_categorical_accuracy: 0.7967
Epoch 19/40
4800/4800 [==============================] - 3s 727us/step - loss: 0.3932 - categorical_accuracy: 0.8667 - val_loss: 0.6061 - val_cat

[85.708333333333329, 89.208333333333329, 87.083333333333329]

----------
## **Regularized 64 GRU**
----------

In [18]:
PARAM_NUM_EPOCHS = 80
PARAM_BATCH_SIZE = 300

from models.regularized_64_gru import Regularized64GRU
from utils.evaluation import cross_validate_model

mymodel = Regularized64GRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3, RANDOM_STATE)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/80
4800/4800 [==============================] - 2s 454us/step - loss: 2.2539 - categorical_accuracy: 0.1415 - val_loss: 2.1930 - val_categorical_accuracy: 0.1650
Epoch 2/80
4800/4800 [==============================] - 1s 258us/step - loss: 2.1493 - categorical_accuracy: 0.1746 - val_loss: 2.1646 - val_categorical_accuracy: 0.1908
Epoch 3/80
4800/4800 [==============================] - 1s 257us/step - loss: 1.9502 - categorical_accuracy: 0.2437 - val_loss: 1.9016 - val_categorical_accuracy: 0.3521
Epoch 4/80
4800/4800 [==============================] - 1s 267us/step - loss: 1.7433 - categorical_accuracy: 0.3321 - val_loss: 1.6424 - val_categorical_accuracy: 0.4142
Epoch 5/80
4800/4800 [==============================] - 1s 259us/step - loss: 1.5845 - categorical_accuracy: 0.4027 - val_loss: 1.5029 - val_categorical_accuracy: 0.4733
Epoch 6/80
4800/4800 [==========

4800/4800 [==============================] - 1s 266us/step - loss: 0.3177 - categorical_accuracy: 0.8990 - val_loss: 0.7740 - val_categorical_accuracy: 0.8217
Epoch 49/80
4800/4800 [==============================] - 1s 279us/step - loss: 0.3300 - categorical_accuracy: 0.8913 - val_loss: 0.7683 - val_categorical_accuracy: 0.8200
Epoch 50/80
4800/4800 [==============================] - 1s 264us/step - loss: 0.3414 - categorical_accuracy: 0.8842 - val_loss: 0.8191 - val_categorical_accuracy: 0.8100
Epoch 51/80
4800/4800 [==============================] - 1s 262us/step - loss: 0.3140 - categorical_accuracy: 0.9027 - val_loss: 0.7603 - val_categorical_accuracy: 0.8317
Epoch 52/80
4800/4800 [==============================] - 1s 268us/step - loss: 0.3147 - categorical_accuracy: 0.8954 - val_loss: 0.7688 - val_categorical_accuracy: 0.8092
Epoch 53/80
4800/4800 [==============================] - 1s 263us/step - loss: 0.2977 - categorical_accuracy: 0.8965 - val_loss: 0.7886 - val_categorical_acc

Epoch 15/80
4800/4800 [==============================] - 1s 263us/step - loss: 0.9617 - categorical_accuracy: 0.6644 - val_loss: 0.7830 - val_categorical_accuracy: 0.7296
Epoch 16/80
4800/4800 [==============================] - 1s 259us/step - loss: 0.9459 - categorical_accuracy: 0.6625 - val_loss: 0.7112 - val_categorical_accuracy: 0.7563
Epoch 17/80
4800/4800 [==============================] - 1s 262us/step - loss: 0.9005 - categorical_accuracy: 0.6935 - val_loss: 0.6921 - val_categorical_accuracy: 0.7554
Epoch 18/80
4800/4800 [==============================] - 1s 261us/step - loss: 0.8788 - categorical_accuracy: 0.7000 - val_loss: 0.7176 - val_categorical_accuracy: 0.7333
Epoch 19/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.8495 - categorical_accuracy: 0.7123 - val_loss: 0.7027 - val_categorical_accuracy: 0.7479
Epoch 20/80
4800/4800 [==============================] - 1s 260us/step - loss: 0.8162 - categorical_accuracy: 0.7260 - val_loss: 0.6542 - val_cat

Epoch 63/80
4800/4800 [==============================] - 1s 269us/step - loss: 0.4026 - categorical_accuracy: 0.8690 - val_loss: 0.4155 - val_categorical_accuracy: 0.8633
Epoch 64/80
4800/4800 [==============================] - 1s 268us/step - loss: 0.4029 - categorical_accuracy: 0.8738 - val_loss: 0.4375 - val_categorical_accuracy: 0.8625
Epoch 65/80
4800/4800 [==============================] - 1s 270us/step - loss: 0.3933 - categorical_accuracy: 0.8690 - val_loss: 0.3974 - val_categorical_accuracy: 0.8750
Epoch 66/80
4800/4800 [==============================] - 1s 268us/step - loss: 0.4028 - categorical_accuracy: 0.8756 - val_loss: 0.4093 - val_categorical_accuracy: 0.8679
Epoch 67/80
4800/4800 [==============================] - 1s 271us/step - loss: 0.3930 - categorical_accuracy: 0.8692 - val_loss: 0.4063 - val_categorical_accuracy: 0.8688
Epoch 68/80
4800/4800 [==============================] - 1s 271us/step - loss: 0.3854 - categorical_accuracy: 0.8785 - val_loss: 0.4159 - val_cat

Epoch 30/80
4800/4800 [==============================] - 1s 260us/step - loss: 0.5726 - categorical_accuracy: 0.8031 - val_loss: 0.7534 - val_categorical_accuracy: 0.7713
Epoch 31/80
4800/4800 [==============================] - 1s 257us/step - loss: 0.5600 - categorical_accuracy: 0.8083 - val_loss: 0.7069 - val_categorical_accuracy: 0.7796
Epoch 32/80
4800/4800 [==============================] - 1s 265us/step - loss: 0.5389 - categorical_accuracy: 0.8212 - val_loss: 0.6787 - val_categorical_accuracy: 0.7833
Epoch 33/80
4800/4800 [==============================] - 1s 260us/step - loss: 0.5486 - categorical_accuracy: 0.8154 - val_loss: 0.6512 - val_categorical_accuracy: 0.7879
Epoch 34/80
4800/4800 [==============================] - 1s 260us/step - loss: 0.5441 - categorical_accuracy: 0.8217 - val_loss: 0.6837 - val_categorical_accuracy: 0.7758
Epoch 35/80
4800/4800 [==============================] - 1s 258us/step - loss: 0.5106 - categorical_accuracy: 0.8256 - val_loss: 0.6633 - val_cat

Epoch 78/80
4800/4800 [==============================] - 1s 269us/step - loss: 0.2471 - categorical_accuracy: 0.9215 - val_loss: 0.5807 - val_categorical_accuracy: 0.8529
Epoch 79/80
4800/4800 [==============================] - 1s 267us/step - loss: 0.2397 - categorical_accuracy: 0.9221 - val_loss: 0.5804 - val_categorical_accuracy: 0.8525
Epoch 80/80
2400/2400 [==============================] - 1s 503us/step
categorical_accuracy: 85.17%
85.88% (+/- 1.69%)


[84.25, 88.208333333333329, 85.166666666666671]